# TPC Async QC - Distortion Calib v1 - LHC22p apass4 vs apass3
## Overview
**Links:**
> Production JIRA: https://alice.its.cern.ch/jira/browse/O2-3701 <br />
> Preliminary information on the runs: <br />
> https://docs.google.com/spreadsheets/d/14vXFYVx3oVE8wgJKmIBhjvAt6NpGp7D6H4AmBM9E0Cw/edit#gid=29745819

**Settings:**
+ B = -0.5 T
+ beam = pp
+ energy (sqrt) = 13.6 TeV

**General notes:**
+ Scan runs
  + No previous apass to compare to
+ Long tracks in general (similar to what is seen in LHC22m apass4)
  + New spike at nClusters = 152
  + Almost looks like an overflow bin (altough there are tracks with nClusters>152)
  + Localized to the middle of sectors
  + Spike distinguised in pT distribution (pT ~500 MeV/c), but observed across full eta
  + Effect not evident in dEdx plots
+ Asymmetry between sides in run 528675
  + Evident in eta plots

**Runs:**
| Period | Run number | FTV0X rate | Duration | TPC Quality | Comment |
|:---:|:---:|:---:|:---:|:---:|:---:|
| LHC22p |528675|2918|00:26:24|`GOOD`|  |
| LHC22p |528699|2892|00:47:22|`GOOD`|  |

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1">Overview</a></span><ul class="toc-item"><li><span><a href="#Tracks" data-toc-modified-id="Tracks-1.1">Tracks</a></span><ul class="toc-item"><li><span><a href="#Phi-Distributions" data-toc-modified-id="Phi-Distributions-1.1.1">Phi Distributions</a></span></li><li><span><a href="#Eta-distribution" data-toc-modified-id="Eta-distribution-1.1.2">Eta distribution</a></span></li><li><span><a href="#Transverse-Momentum-(pT)" data-toc-modified-id="Transverse-Momentum-(pT)-1.1.3">Transverse Momentum (pT)</a></span></li><li><span><a href="#Sign-of-tracks" data-toc-modified-id="Sign-of-tracks-1.1.4">Sign of tracks</a></span></li><li><span><a href="#Charge-over-transverse-momentum-(q/pT)" data-toc-modified-id="Charge-over-transverse-momentum-(q/pT)-1.1.5">Charge over transverse momentum (q/pT)</a></span></li><li><span><a href="#Number-of-clusters" data-toc-modified-id="Number-of-clusters-1.1.6">Number of clusters</a></span></li><li><span><a href="#Number-of-clusters-vs-phi" data-toc-modified-id="Number-of-clusters-vs-phi-1.1.7">Number of clusters vs phi</a></span></li><li><span><a href="#Number-of-clusters-vs-eta" data-toc-modified-id="Number-of-clusters-vs-eta-1.1.8">Number of clusters vs eta</a></span></li><li><span><a href="#Number-of-clusters-vs-pT" data-toc-modified-id="Number-of-clusters-vs-pT-1.1.9">Number of clusters vs pT</a></span></li></ul></li><li><span><a href="#PID" data-toc-modified-id="PID-1.2">PID</a></span><ul class="toc-item"><li><span><a href="#dEdxTot-of-MIPs" data-toc-modified-id="dEdxTot-of-MIPs-1.2.1">dEdxTot of MIPs</a></span></li><li><span><a href="#Number-of-clusters-PID" data-toc-modified-id="Number-of-clusters-PID-1.2.2">Number of clusters PID</a></span></li><li><span><a href="#MIP-dEdxTot-vs-Sector-(vs-Phi-but-both-sides)" data-toc-modified-id="MIP-dEdxTot-vs-Sector-(vs-Phi-but-both-sides)-1.2.3">MIP dEdxTot vs Sector (vs Phi but both sides)</a></span></li><li><span><a href="#MIP-dEdxTot-vs-Number-of-Clusters" data-toc-modified-id="MIP-dEdxTot-vs-Number-of-Clusters-1.2.4">MIP dEdxTot vs Number of Clusters</a></span></li><li><span><a href="#MIP-dEdxTot-vs-sin(phi)" data-toc-modified-id="MIP-dEdxTot-vs-sin(phi)-1.2.5">MIP dEdxTot vs sin(phi)</a></span></li><li><span><a href="#MIP-dEdxTot-vs-tan(lambda)" data-toc-modified-id="MIP-dEdxTot-vs-tan(lambda)-1.2.6">MIP dEdxTot vs tan(lambda)</a></span></li><li><span><a href="#dEdx-Tot-vs-momentum-(p)" data-toc-modified-id="dEdx-Tot-vs-momentum-(p)-1.2.7">dEdx Tot vs momentum (p)</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
from IPython.display import display, Markdown, HTML
import ipywidgets as widgets
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.checkHistograms import *

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))

In [ ]:
# Read the Root Files
path = "/cave/alice/data/2022/tpc_distortion/apass4/"
passName = "apass4"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
fileList = fileList[3:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
runList

### Tracks
#### Phi Distributions

In [ ]:
%jsroot on
objectName="hPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME L",
                                                              maxColumns=1,grid=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,histogram[i].GetMinimum(),0,histogram[i].GetMaximum())
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,histogram[i].GetMinimum(),(j*2*math.pi)/18,histogram[i].GetMaximum())
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,histogram[i].GetMaximum(),str(j))
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

In [ ]:
%jsroot on
objectName="hPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME L",
                                                              maxColumns=1,grid=False)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,histogram[i].GetMinimum(),0,histogram[i].GetMaximum())
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,histogram[i].GetMinimum(),(j*2*math.pi)/18,histogram[i].GetMaximum())
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,histogram[i].GetMaximum(),str(j))
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Eta distribution

In [ ]:
%jsroot on
objectName="hEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,xAxisRange=[-1.1,1.1],legendNames=runList,
                                                              pads=True,drawOption="SAME L",
                                                              maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Transverse Momentum (pT)

In [ ]:
%jsroot on
objectName="hPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,yAxisRange=[1e-5,40e-3],legendNames=runList,
                                                              pads=True,drawOption="SAME L",
                                                              log="logxy",maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Sign of tracks

In [ ]:
%jsroot on
objectName="hSign"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME HIST",
                                                              maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Charge over transverse momentum (q/pT)

In [ ]:
%jsroot on
objectName="hQOverPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME HIST",
                                                              maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Number of clusters

In [ ]:
%jsroot on
objectName="hNClustersAfterCuts"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,xAxisRange=[50,160],yAxisRange=[0,55e-3],legendNames=runList,
                                                              pads=True,drawOption="SAME L",
                                                              maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Number of clusters vs phi

In [ ]:
%jsroot on
objectName="h2DNClustersPhiAside"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList, yAxisRange = [50,170],
                                                              pads=True, drawOption="COLZ", maxColumns=1)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,50,0,170)
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,50,(j*2*math.pi)/18,170)
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,55,str(j))
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

In [ ]:
%jsroot on
objectName="h2DNClustersPhiCside"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList, yAxisRange = [50,170],
                                                              pads=True, drawOption="COLZ", maxColumns=1)
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine(0,50,0,170)
        sectorBoundary.SetLineStyle(2)
        sectorNum = ROOT.TText(.5,.5,"test")
        sectorNum.SetTextAlign(22)
        sectorNum.SetTextColor(ROOT.kGray+1)
        sectorNum.SetTextFont(0)
        for j in range(19):
            sectorBoundary.DrawLine((j*2*math.pi)/18,50,(j*2*math.pi)/18,170)
            if j < 18 : sectorNum.DrawText(((j+0.5)*2*math.pi)/18,55,str(j))
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### Number of clusters vs eta

In [ ]:
%jsroot on
objectName="h2DNClustersEta"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList, xAxisRange = [-1.1,1.1], yAxisRange = [50,170],
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")


for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### Number of clusters vs pT

In [ ]:
%jsroot on
objectName="h2DNClustersPt"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList, yAxisRange = [50,170],
                                                              log="logx",pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")


for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

### PID

#### dEdxTot of MIPs

In [ ]:
%jsroot on
objectName="hdEdxTotMIP_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME HIST",
                                                              log="logx",yAxisRange=[0,0.12],maxColumns=2)
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        sectorBoundary = ROOT.TLine()
        sectorBoundary.SetLineStyle(1)
        sectorBoundary.SetLineColor(ROOT.kGray+2)
        sectorBoundary.SetLineWidth(2)
        sectorBoundary.DrawLine(50,0,50,0.12)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### Number of clusters PID

In [ ]:
%jsroot on
objectName="hNClsPID"
[hist,legend,canvas,pad1] = drawHistograms(objectName,rootDataFile,normalize=True,
                                                              legend=False,legendNames=runList,
                                                              pads=True,drawOption="SAME HIST",
                                                              xAxisRange=[50,160],yAxisRange=[0,0.065])
leg = ROOT.TLegend()
leg.AddEntry(hist[0], passName, "lep")

#Sector boundatries
for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex==0 :
            leg.Draw()
canvas.cd()
canvas.Draw()

#### MIP dEdxTot vs Sector (vs Phi but both sides)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSec_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList,log="logz",
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs Number of Clusters

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsNcl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList, xAxisRange=[60,160],log="logz",
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs sin(phi)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsSnp_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList,log="logz",
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### MIP dEdxTot vs tan(lambda)

In [ ]:
%jsroot on
objectName="hdEdxTotMIPVsTgl_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList,log="logz",
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()

#### dEdx Tot vs momentum (p)

In [ ]:
%jsroot on
objectName="hdEdxTotVsP_TPC"
[hist,legend,canvas,pad1] = drawHistograms(objectName, rootDataFile, normalize=True,
                                                              legend=False, legendNames=runList,log="logxyz",
                                                              pads=True, drawOption="COLZ")
leg1 = ROOT.TLegend()
leg1.AddEntry(hist[0], passName, "p")

for i in range(len(rootDataFile)):
    padRun = pad1.cd(i+1)
    for typeIndex,histogram in enumerate([hist]):
        padRun.cd(typeIndex+1)
        if typeIndex == 0 : leg1.Draw()
        elif typeIndex == 1 : leg2.Draw()
canvas.Draw()